In [1]:
# Install the package
!pip install spanish_sentiment_analysis

     |████████████████████████████████| 15.8 MB 147 kB/s 
     |████████████████████████████████| 1.2 MB 44.5 MB/s 


In [2]:
import pandas as pd
import re
from google.colab import files
from google.colab import drive
from classifier import SentimentClassifier
import io

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Will download some functions from the nltk package if not found on the computer
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# Upload the file
uploaded = files.upload()

Saving Corrected_Final_All.csv to Corrected_Final_All.csv


In [4]:
# Define the dataframe
df = pd.read_csv(io.BytesIO(uploaded['Corrected_Final_All.csv']))
df

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags
0,0,2021-03-27T04:09:42+00:00,1.380000e+18,1.380000e+18,@Diputado_Canelo Hagamos otro por el uno de ma...,es,False,False,0.0,1.0,0.0,0.0,NaN,NaN,NaN,9.094314e+08,carmeno1635,🌻🌺Carmen Ochoa...🌺🌻,False,1.380000e+18,1.290000e+18,#1mayo,NaN,NaN,"[{""url"": ""https://pbs.twimg.com/media/ExdUpC3X...",2021-03-27,"['hacer', 'mayo', 'cazar', 'fantasma', 'mayo']",[''],[':/'],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['hacer', 'mayo', 'cazar', 'fantasma']"
1,1,2021-03-22T21:12:09+00:00,1.370000e+18,1.370000e+18,Después de esperar con ancias el #28F ahora es...,es,False,False,1.0,4.0,0.0,0.0,NaN,NaN,NaN,1.260000e+18,olga73427503,olga Lidia 👄💄🌹,False,NaN,NaN,"#28F , #1Mayo , #15Septiembre","nayibbukele , waraujo64",NaN,[],2021-03-22,"['despues', 'esperar', 'ancia', 'ahora', 'espe...",['💙🤍💙'],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['despues', 'esperar', 'ancia', 'ahora', 'espe..."
2,2,2021-03-22T12:30:53+00:00,1.370000e+18,1.370000e+18,Espero que ésto llegue hasta oídos de la nueva...,es,False,False,0.0,1.0,1.0,0.0,NaN,NaN,NaN,1.350000e+18,Mr_DonNadie,Dios del Olimpo 🇸🇻⚖️ UES,False,NaN,NaN,"#1Mayo , #BecasUniversitarias",AsambleaSV,NaN,[],2021-03-22,"['esperar', 'llegar', 'oido', 'nuevo', 'inicia...",[''],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['esperar', 'llegar', 'oido', 'nuevo', 'inicia..."
3,3,2021-04-04T12:56:55+00:00,1.380000e+18,1.380000e+18,A menos de un mes del #1Mayo Urkullu teme perd...,es,False,False,3.0,5.0,0.0,0.0,NaN,NaN,NaN,1.662306e+08,Antifaxismoa,Sare Antifaxista,False,NaN,NaN,"#1Mayo , #pandemia , #oasisvasco , #regimenjel...",NaN,NaN,[],2021-04-04,"['menos', 'mes', 'mayo', 'urkullu', 'temer', '...",[''],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['menos', 'mes', 'urkullu', 'temer', 'perder',..."
4,4,2021-04-03T20:14:57+00:00,1.380000e+18,1.380000e+18,La X Edición del Festival Internacional Un Pue...,es,False,False,1.0,3.0,0.0,1.0,NaN,NaN,NaN,1.320000e+18,Radiometrohaba1,Radio Metropolitana Cuba,False,NaN,NaN,"#LaHabana , #1Mayo , #MiHabanaEsMúsica , #Cuba...","AneliPupo , BenitezCordovi , LoureiroMilan , n...",https://t.co/caGq26zE1K,"[{""url"": ""https://pbs.twimg.com/media/EyE0s-AW...",2021-04-03,"['edicion', 'festival', 'internacional', 'puen...",[''],"[':/', ':/']",Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['edicion', 'festival', 'internacional', 'puen..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22155,22155,2021-08-03 12:01:04,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-03,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load..."
22156,22156,2021-08-05 22:02:30,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-05,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're..."
22157,22157,2021-08-06 17:19:38,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-06,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're..."
221

In [5]:
# Define the classifier
clf = SentimentClassifier()

In [6]:
# Test
x = "Esta muy buena esa pelicula"

print(x + ' ==> %.5f' % clf.predict(x))

Esta muy buena esa pelicula ==> 0.93136


In [7]:
df['lang_langdetect'].value_counts()

es    2317
en    1015
pt      50
sl      35
da      32
hr      21
it      19
ca      18
fr      18
id      15
tl      11
de       9
et       8
so       6
af       5
lt       5
cy       5
fi       4
ro       4
hu       3
sk       3
nl       3
lv       3
vi       2
no       2
tr       1
sq       1
sw       1
pl       1
Name: lang_langdetect, dtype: int64

In [8]:
df['langTb'].value_counts()

es    315
en    256
bs      4
pt      2
mi      1
hr      1
et      1
it      1
hu      1
ja      1
fi      1
hi      1
Name: langTb, dtype: int64

In [9]:
df['lang'].value_counts()

es    16690
en     5470
Name: lang, dtype: int64

As there are many values missing from the language validation, I'm gonna use the 'lang' column

In [10]:
df = df[df['lang']=='es']

Sample the data to test the classifier

In [12]:
df1 =df.sample(200)
df1

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags
2775,2775,2021-07-23T22:16:21+00:00,1.420000e+18,1.420000e+18,El nuevo uniforme de la p0licía con un diminut...,es,False,False,0.0,0.0,0.0,0.0,1.420000e+18,NaN,NaN,4.006303e+07,somosamb,SomosAmb🌾,False,NaN,NaN,#ColombiaResiste,NaN,NaN,[],2021-07-23,"['nuevo', 'uniforme', 'licio', 'diminuto', 'co...",[''],[],olombiaResiste_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['nuevo', 'uniforme', 'licio', 'diminuto', 'co..."
175,175,2021-05-02T23:31:24+00:00,1.390000e+18,1.390000e+18,"Leales siempre, traidores nunca... Cada dia so...",es,False,False,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.350000e+18,TaniaAl89297299,Tania Alejandra Torrez,False,NaN,NaN,"#MayoDignidadNacionalVencemos , #Nicaragua , #...",NaN,NaN,"[{""url"": ""https://pbs.twimg.com/media/E0a3xR3W...",2021-05-02,"['leal', 'siempre', 'traidor', 'nunca', 'cada'...",[''],[':/'],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['leal', 'siempre', 'traidor', 'nunca', 'cada'..."
4958,4958,2021-06-27T14:53:36+00:00,1.410000e+18,1.410000e+18,Ahora todo tiene sentido :v ajajajajajajajajaj...,es,False,False,0.0,0.0,0.0,0.0,NaN,NaN,NaN,3.302838e+09,santiagofiore8,J. Santiago G.,False,NaN,NaN,"#GobiernoDeLaMuerte , #ElParoNoPara",GustavoBolivar,NaN,"[{""url"": ""https://pbs.twimg.com/media/E45aSFFX...",2021-06-27,"['ahora', 'ajajajajajajajajajj', 'gobiernodela...",[''],[],lParoNOPara_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['ahora', 'ajajajajajajar']"
15629,15629,2021-05-19 1:01:52,NaN,NaN,Me atrevo a decir que la gran mayoría de perso...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-19,"['atreverse', 'decir', 'gran', 'mayoria', 'per...",[''],[],NaN,Reddit,es,15.0,NaN,es,"['atreverse', 'decir', 'gran', 'mayoria', 'per..."
1472,1472,2021-05-23T00:59:30+00:00,1.400000e+18,1.400000e+18,@EsDeOrates #SOSZapateiroVioladorDDHH\n#@Franc...,es,False,False,0.0,1.0,0.0,0.0,NaN,NaN,NaN,2.251052e+08,mlrb68,Eterna inconforme,False,1.400000e+18,303081415.0,"#SOSZapateiroVioladorDDHH , #SOSColombiaDDHH ,...","EsDeOrates , WOLA_org , cvoule , ONUHumanRight...",NaN,[],2021-05-23,"['soszapateirovioladorddhh', 'soscolombiaddhh'...",[''],[],olombiaAlertaRoja_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5986,5986,2021-05-18T16:16:53+00:00,1.390000e+18,1.390000e+18,@fertome2 Esto va de mal en peor.\nA ese #Terr...,es,False,False,0.0,0.0,0.0,0.0,NaN,NaN,NaN,4.152596e+08,mafaldafox,Charles De Gaulle 🇨🇴,False,1.390000e+18,59637309.0,"#TerrorismoUrbano , #ParoDestructorSOS , #Comi...","fertome2 , SenadoGovCo , ConcejoDeBogota , ONU...",NaN,[],2021-05-18,"['ir', 'mal', 'peor', 'terrorismourbano', 'com...",[''],[],aroDestructorSOS_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['ir', 'mal', 'peor', 'combatir', 'él', 'todo'..."
8989,8989,2021-08-18T21:00:44+00:00,1.430000e+18,1.430000e+18,Secretario de Marina ofrece disculpas a Poder ...,es,False,False,0.0,1.0,0.0,0.0,NaN,NaN,NaN,1.300000e+18,esprimeralinea,Primera Línea Mx,False,NaN,NaN,#PrimeraLínea,NaN,https://t.co/eI3qbSA28D,[],2021-08-18,"['secretario', 'marino', 'ofrecer', 'disculpa'...",['📢'],"[':/', '8D']",rimeraLinea_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['secretario', 'marino', 'ofrecer', 'disculpa'..."
7467,7467,2021-07-18T22:26:52+00:00,1.420000e+18,1.420000e+18,#SOSAsambleaPopular \n#soscolombiadddhh \n#Ref...,es,False,False,2.0,1.0,1.0,1

In [13]:
# Apply the classifier to the column of the preprocessed data
x = df1['preprocessed_data_without_hashtags'].apply(lambda x: clf.predict(x))

In [14]:
# Add the results to the dataframe
df1['sentiment']= x

In [15]:
df1[['full_text','preprocessed_data_without_hashtags','sentiment']].head()

,full_text,preprocessed_data_without_hashtags,sentiment
2775,El nuevo uniforme de la p0licía con un diminut...,"['nuevo', 'uniforme', 'licio', 'diminuto', 'co...",0.055590
175,"Leales siempre, traidores nunca... Cada dia so...","['leal', 'siempre', 'traidor', 'nunca', 'cada'...",0.214566
4958,Ahora todo tiene sentido :v ajajajajajajajajaj...,"['ahora', 'ajajajajajajar']",0.167089
15629,Me atrevo a decir que la gran mayoría de perso...,"['atreverse', 'decir', 'gran', 'mayoria', 'per...",0.009526
1472,@EsDeOrates #SOSZapateiroVioladorDDHH\n#@Franc...,[],0.541333


Doing some preprocessing to the full text but keeping some original structure, to see if there are differences

In [16]:
def remove_mention_text(text):
    mention_pattern = r'@\S+'
    return re.sub(pattern=mention_pattern, repl=' ', string=text)

def remove_hashtags(text):
    hashtag_pattern = r'#\S+'
    return re.sub(pattern=hashtag_pattern, repl=' ', string=text)

def removing_urls(text):
    return re.sub(r'https?://\S+|www\.\S+','',text)

In [17]:
df1['full_text'] = df1['full_text'].apply(lambda x: remove_mention_text(x))
df1['full_text'] = df1['full_text'].apply(lambda x: remove_hashtags(x))
df1['full_text'] = df1['full_text'].apply(lambda x: removing_urls(x))

In [18]:
# Apply the classifier to the full text without preprocessing
df1['sentiment_full'] = df1['full_text'].apply(lambda x: clf.predict(x))

In [19]:
df1[['full_text','preprocessed_data_without_hashtags','sentiment', 'sentiment_full']].head(20)

,full_text,preprocessed_data_without_hashtags,sentiment,sentiment_full
2775,El nuevo uniforme de la p0licía con un diminut...,"['nuevo', 'uniforme', 'licio', 'diminuto', 'co...",0.055590,0.009384
175,"Leales siempre, traidores nunca... Cada dia so...","['leal', 'siempre', 'traidor', 'nunca', 'cada'...",0.214566,0.115354
4958,Ahora todo tiene sentido :v ajajajajajajajajaj...,"['ahora', 'ajajajajajajar']",0.167089,0.512684
15629,Me atrevo a decir que la gran mayoría de perso...,"['atreverse', 'decir', 'gran', 'mayoria', 'per...",0.009526,0.000191
1472,\n# \n \n \n \n \n \n \n \n \n \n \n \n ...,[],0.541333,0.541333
1098,Así de grave será la situación en Colombia...,"['asi', 'grave', 'ser', 'situacion', 'colombio...",0.047310,0.024613
1701,\n# \n \n \n \n \n \n \...,[],0.541333,0.541333
6976,manifestaciones generan afectación vial sobre ...,"['manifestación', 'generar', 'afectacion', 'vi...",0.261116,0.566028
11861,"Que marche al lado de sus adoctrinados, los ...","['marchar', 'lado', 'adoctrinado', 'verdadero'...",0.034581,0.079653
9128,Parece que es cierto que soltaron a 28 delincu...,"['parecer', 'cierto', 'soltar', 'delincuente',...",0.020109,0.005556


In [35]:
# Error between "sentiment with preprocessed text" and "sentiment with full text"
df1['percent_error']= (abs(df1['sentiment']-df1['sentiment_full']))*100/df1['sentiment']

In [36]:
df2 = df1[['full_text','preprocessed_data_without_hashtags','sentiment', 'sentiment_full','percent_error']]
df2

,full_text,preprocessed_data_without_hashtags,sentiment,sentiment_full,percent_error
2775,El nuevo uniforme de la p0licía con un diminut...,"['nuevo', 'uniforme', 'licio', 'diminuto', 'co...",0.055590,0.009384,83.118334
175,"Leales siempre, traidores nunca... Cada dia so...","['leal', 'siempre', 'traidor', 'nunca', 'cada'...",0.214566,0.115354,46.238578
4958,Ahora todo tiene sentido :v ajajajajajajajajaj...,"['ahora', 'ajajajajajajar']",0.167089,0.512684,206.832794
15629,Me atrevo a decir que la gran mayoría de perso...,"['atreverse', 'decir', 'gran', 'mayoria', 'per...",0.009526,0.000191,97.999960
1472,\n# \n \n \n \n \n \n \n \n \n \n \n \n ...,[],0.541333,0.541333,0.000000
...,...,...,...,...,...
5986,Esto va de mal en peor.\nA ese hay q comba...,"['ir', 'mal', 'peor', 'combatir', 'él', 'todo'...",0.203528,0.040186,80.255456
8989,Secretario de Marina ofrece disculpas a Poder ...,"['secretario', 'marino', 'ofrecer', 'disculpa'...",0.264838,0.384432,45.157773
7467,\n \n \n \n \n \n\n \n\n con estos vanda...,['vandalo'],0.013782,0.541333,3827.725536
15215,"Leí la parte de ""instaurar régimen comunista ""...","['lei', 'parte', 'instaurar', 'regimen', 'comu...",0.015981,0.017642,10.393876


In [ ]:
df1[['full_text','preprocessed_data_without_hashtags','sentiment', 'sentiment_full', 'percent_error']].to_excel('Spanish_sample.xlsx', index=False)

In [37]:
df2[df2['percent_error']>100]

,full_text,preprocessed_data_without_hashtags,sentiment,sentiment_full,percent_error
4958,Ahora todo tiene sentido :v ajajajajajajajajaj...,"['ahora', 'ajajajajajajar']",0.167089,0.512684,206.832794
6976,manifestaciones generan afectación vial sobre ...,"['manifestación', 'generar', 'afectacion', 'vi...",0.261116,0.566028,116.772772
11861,"Que marche al lado de sus adoctrinados, los ...","['marchar', 'lado', 'adoctrinado', 'verdadero'...",0.034581,0.079653,130.336346
4704,"Nos acompañó ola Pazifica Ong, el partido CO...","['acompano', 'ola', 'pazifico', 'ong', 'partid...",0.048314,0.119546,147.435445
1793,Se indignan qnd queman un CAI pero no qnd lo u...,"['indignar', 'qnd', 'quemar', 'cai', 'qnd', 'u...",0.001508,0.004800,218.248708
11205,Señora De Rivero: solicitamos que en esta coyu...,"['senora', 'rivero', 'solicitar', 'coyuntura',...",0.050696,0.192724,280.159985
15872,Excelentes consejos. Yo hice el mío hace 21 añ...,"['excelente', 'consejo', 'hacer', 'mio', 'hace...",0.163792,0.511477,212.272816
9538,"No me dejaría ni saludar d duque, incluso le...","['dejarir', 'saludar', 'duque', 'incluso', 'ne...",0.008327,0.023288,179.655389
8999,México recibe primeras solicitudes de refugio ...,"['mexico', 'recibir', 'primero', 'solicitud', ...",0.146069,0.624107,327.269253
8097,Me duele demasiado y lloró bastante por lo...,"['doler', 'demasiado', 'llorar', 'bastante', '...",0.050330,0.133867,165.978910


In [40]:
df2.loc[9713, : ][0]

'      \nNo hable de paz fuera del país, mientras acá en menos de 12 horas, una masacre☝️ y dos asesinatos, firmante de paz y jóven de primera línea.\n '

There are some cases where the classification with the "full_text" is better than the one made with "preprocessed_data_without_hashtags", but this does not represent a majority, at least not in the sample taken.

In [42]:
# Apply the classifier to the full text without preprocessing
df['sentiment'] = df['preprocessed_data_without_hashtags'].apply(lambda x: clf.predict(x))

In [43]:
df

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment
0,0,2021-03-27T04:09:42+00:00,1.380000e+18,1.380000e+18,@Diputado_Canelo Hagamos otro por el uno de ma...,es,False,False,0.0,1.0,0.0,0.0,NaN,NaN,NaN,9.094314e+08,carmeno1635,🌻🌺Carmen Ochoa...🌺🌻,False,1.380000e+18,1.290000e+18,#1mayo,NaN,NaN,"[{""url"": ""https://pbs.twimg.com/media/ExdUpC3X...",2021-03-27,"['hacer', 'mayo', 'cazar', 'fantasma', 'mayo']",[''],[':/'],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['hacer', 'mayo', 'cazar', 'fantasma']",0.259479
1,1,2021-03-22T21:12:09+00:00,1.370000e+18,1.370000e+18,Después de esperar con ancias el #28F ahora es...,es,False,False,1.0,4.0,0.0,0.0,NaN,NaN,NaN,1.260000e+18,olga73427503,olga Lidia 👄💄🌹,False,NaN,NaN,"#28F , #1Mayo , #15Septiembre","nayibbukele , waraujo64",NaN,[],2021-03-22,"['despues', 'esperar', 'ancia', 'ahora', 'espe...",['💙🤍💙'],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['despues', 'esperar', 'ancia', 'ahora', 'espe...",0.180461
2,2,2021-03-22T12:30:53+00:00,1.370000e+18,1.370000e+18,Espero que ésto llegue hasta oídos de la nueva...,es,False,False,0.0,1.0,1.0,0.0,NaN,NaN,NaN,1.350000e+18,Mr_DonNadie,Dios del Olimpo 🇸🇻⚖️ UES,False,NaN,NaN,"#1Mayo , #BecasUniversitarias",AsambleaSV,NaN,[],2021-03-22,"['esperar', 'llegar', 'oido', 'nuevo', 'inicia...",[''],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['esperar', 'llegar', 'oido', 'nuevo', 'inicia...",0.179048
3,3,2021-04-04T12:56:55+00:00,1.380000e+18,1.380000e+18,A menos de un mes del #1Mayo Urkullu teme perd...,es,False,False,3.0,5.0,0.0,0.0,NaN,NaN,NaN,1.662306e+08,Antifaxismoa,Sare Antifaxista,False,NaN,NaN,"#1Mayo , #pandemia , #oasisvasco , #regimenjel...",NaN,NaN,[],2021-04-04,"['menos', 'mes', 'mayo', 'urkullu', 'temer', '...",[''],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['menos', 'mes', 'urkullu', 'temer', 'perder',...",0.009807
4,4,2021-04-03T20:14:57+00:00,1.380000e+18,1.380000e+18,La X Edición del Festival Internacional Un Pue...,es,False,False,1.0,3.0,0.0,1.0,NaN,NaN,NaN,1.320000e+18,Radiometrohaba1,Radio Metropolitana Cuba,False,NaN,NaN,"#LaHabana , #1Mayo , #MiHabanaEsMúsica , #Cuba...","AneliPupo , BenitezCordovi , LoureiroMilan , n...",https://t.co/caGq26zE1K,"[{""url"": ""https://pbs.twimg.com/media/EyE0s-AW...",2021-04-03,"['edicion', 'festival', 'internacional', 'puen...",[''],"[':/', ':/']",Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['edicion', 'festival', 'internacional', 'puen...",0.612892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16685,16685,2021-08-26 17:13:16,NaN,NaN,Por que proponen soluciones peores que los pro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-26,"['proponer', 'solución', 'peor', 'problema', '...",[''],[],NaN,Reddit,es,-8.0,NaN,es,"['proponer', 'solución', 'peor', 'problema', '...",0.006783
16686,16686,2021-08-26 21:16:19,NaN,NaN,Yo también quisiera ir el mes de septiembre pe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-26,"['tambien', 'querer', 'ir', 'mes', 'septiembre...",[''],[],NaN,Reddit,es,0.0,NaN,es,"['tambien', 'querer', 'ir', 'mes', 'septiembre...",0.030641
16687,16687,2021-08-26 23:26:52,NaN,NaN,"No existe ""ilegalidad"" en el movimiento, el pr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-26,"['existir', 'ilegalidad', 'movimiento', 'probl...",[''],[],Na

In [44]:
df.to_csv('SentimentAnalysis_Spanish.csv', index=False)